# Logging进阶篇  

和基础篇比，本篇才算是打开了logging所有的大门，基础篇的很多操作其实内部都做了不少默认动作，尤其是basicConfig，所做的内容有很多，只不过光靠一个basicConfig没有办法好好展现而已。  
本篇从基础介绍了logging 模块的基本类，通过对基础类的实例化来创建日志记录器，虽然相对基础篇更复杂，但其实更加符合python的思维。

logging 底层有四个大类：
* Logger: 也就是日志记录器类，实例化后直接用来操作各种日志记录操作
* Handler: 用于确定将日志记录输出到哪里，一个日志记录器可以有多个handler
* Filter: 更细粒度的区分对什么日志进行输出
* Formatter: 更细粒度的确定日志输出格式

最终日志记录器生成一个 LogRecord 实例，从而完成日志输出

## 整个logging的流程

![LoggingFlow](https://docs.python.org/3/_images/logging_flow.png)

流程图可以表述成以下内容

## Logger  

命名

## handlers

确定最终

## filters

## formatters